In [2]:
%load_ext memory_profiler

In [3]:
%memit

peak memory: 40.32 MiB, increment: 0.07 MiB


In [5]:
%load_ext autoreload
%autoreload 2
 
%matplotlib inline
# Set below to 0 or 1 for GPU
%env CUDA_VISIBLE_DEVICES=1
from __future__ import absolute_import, division, print_function
import json
import os
if os.path.isfile("../code/config.json"):
    with open("../code/config.json", "r") as f:
        config = json.load(f)
else:
    print("Please run setup.py in this directory before running any .ipynb's.")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import tensorflow as tf
import time
import shutil
import cPickle as pickle
from collections import OrderedDict, defaultdict

import sys
sys.path.append("../code")
from utils.bags import *
from utils.learning import *
from multimodal_generative_model import *
from st_graph import *
from data_utils import *
from stg_node import *
from experiment_details import NUM_DATAFILES, ROWS_TO_EXTRACT, EDGE_STATE_COMBINE_METHOD, EDGE_RADIUS

config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=1


{u'catkin_ws_dir': u'/Users/borisi/catkin_ws',
 u'data_dir': u'data',
 u'feature_extraction_dict': u'slim',
 u'julia_pkg_dir': u'/Users/borisi/.julia/v0.6',
 u'models_dir': u'models'}

In [6]:
cvd = %env CUDA_VISIBLE_DEVICES
model_dir = os.path.join(config["models_dir"], ("GPU" + cvd if cvd else "CPU") + "_checkpoints")
# shutil.rmtree(model_dir, ignore_errors=True)

sc = tf.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=False)
rc = tf.estimator.RunConfig().replace(session_config=sc, model_dir=model_dir,
                                      save_summary_steps=100, save_checkpoints_steps=5000,
                                      keep_checkpoint_max=5000, tf_random_seed=None)
rc.environment = None    # required due to a bug in tf.contrib.learn.Experiment.train_and_evaluate
model_dir

u'models/GPU1_checkpoints'

In [7]:
# Edit model hyperparameters if desired, e.g.:
# hps.N = 2
# hps.K = 5
hps.values()

{'GMM_components': 16,
 'K': 5,
 'MLP_dropout_keep_prob': 0.9,
 'MVG_latent_dim': 32,
 'N': 2,
 'alpha': 1,
 'batch_size': 16,
 'dec_GMM_proj_MLP_dims': None,
 'dec_rnn_dim': [128],
 'dec_sample_model_prob_crossover': 20000,
 'dec_sample_model_prob_divisor': 6,
 'dec_sample_model_prob_final': 0.0,
 'dec_sample_model_prob_start': 0.0,
 'enc_rnn_dim_edge': [8],
 'enc_rnn_dim_edge_influence': [8],
 'enc_rnn_dim_future': [32],
 'enc_rnn_dim_history': [32],
 'enc_rnn_dim_multiple_inputs': [8],
 'fuzz_factor': 0.05,
 'grad_clip': 1.0,
 'k': 3,
 'k_eval': 50,
 'kl_crossover': 8000,
 'kl_decay_rate': 0.99995,
 'kl_exact': True,
 'kl_min': 0.07,
 'kl_sigmoid_divisor': 6,
 'kl_weight': 1.0,
 'kl_weight_start': 0.0001,
 'latent_type': 'discrete',
 'learning_decay_rate': 0.9999,
 'learning_rate': 0.001,
 'log_p_yt_xz_max': 50,
 'log_sigma_max': 10,
 'log_sigma_min': -10,
 'min_learning_rate': 1e-05,
 'minimum_history_length': 5,
 'optimizer': tensorflow.python.training.adam.AdamOptimizer,
 'optimi

In [8]:
%memit

peak memory: 191.32 MiB, increment: 0.00 MiB


# Data Loading

In [9]:
NUM_DATAFILES = 2
ROWS_TO_EXTRACT = 100

robot_node = STGNode('Al Horford', 'HomeC')
positions_map_path = "data/positions_map.pkl"
pos_dict_path = "data/pos_dict_%d_files_%s_rows.pkl" % (NUM_DATAFILES, str(ROWS_TO_EXTRACT))

if os.path.isfile(pos_dict_path):
    with open(pos_dict_path, 'rb') as f:
        pos_dict = pickle.load(f)
else:
    pos_dict = get_pos_dict(train_files, 
                            positions_map_path=positions_map_path, 
                            rows_to_extract=ROWS_TO_EXTRACT)
    with open(pos_dict_path, 'wb') as f:
        pickle.dump(pos_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

STG = SpatioTemporalGraphCVAE(pos_dict, robot_node, 
                              edge_radius=EDGE_RADIUS,
                              edge_state_combine_method=EDGE_STATE_COMBINE_METHOD)

train_data_dict_path = "data/train_data_dict_%d_files_%s_rows.pkl" % (NUM_DATAFILES, ROWS_TO_EXTRACT)
if os.path.isfile(train_data_dict_path):
    with open(train_data_dict_path, 'rb') as f:
        train_data_dict = pickle.load(f)
else:
    train_data_dict = get_data_dict(train_files, 
                                    positions_map_path=positions_map_path, 
                                    rows_to_extract=ROWS_TO_EXTRACT)
    with open(train_data_dict_path, 'wb') as f:
        pickle.dump(train_data_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

hps.add_hparam("nodes_standardization", train_data_dict["nodes_standardization"])
hps.add_hparam("extras_standardization", {"mean": train_data_dict["extras_mean"],
                                          "std": train_data_dict["extras_std"]})
hps.add_hparam("labels_standardization", train_data_dict["labels_standardization"])
hps.add_hparam("pred_indices", train_data_dict["pred_indices"])

eval_data_dict_path = "data/eval_data_dict_%d_files_%s_rows.pkl" % (NUM_DATAFILES, str(ROWS_TO_EXTRACT))
if os.path.isfile(eval_data_dict_path):
    with open(eval_data_dict_path, 'rb') as f:
        eval_data_dict = pickle.load(f)
else:
    eval_data_dict = get_data_dict(eval_files, positions_map_path=positions_map_path)
    with open(eval_data_dict_path, 'wb') as f:
        pickle.dump(eval_data_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

train_input_function = tf.estimator.inputs.numpy_input_fn(train_data_dict["input_dict"],
                                                          y = train_data_dict["labels"],
                                                          batch_size = hps.batch_size,
                                                          num_epochs = None,
                                                          shuffle = True)

# Need all possible nodes to have been seen by the STG above, does 
# that mean we feed in the all_files pos_dict in order to create 
# the nodes ahead of time?
token_eval_node = None
token_eval_label_node = None
for node in eval_data_dict["input_dict"]:
    if isinstance(node, STGNode):
        token_eval_node = node
        token_eval_label_node = convert_to_label_node(node)
        break

for node in train_data_dict["input_dict"]:
    if isinstance(node, STGNode):
        if node not in eval_data_dict["input_dict"]:
            eval_data_dict["input_dict"][node] = np.zeros_like(eval_data_dict["input_dict"][token_eval_node])
            eval_data_dict["labels"][convert_to_label_node(node)] = np.zeros_like(eval_data_dict["labels"][token_eval_label_node])

eval_input_function = tf.estimator.inputs.numpy_input_fn(eval_data_dict["input_dict"],
                                                         y = eval_data_dict["labels"],
                                                         batch_size = 4,
                                                         num_epochs = 1,
                                                         shuffle = False)

In [10]:
train_data_dict["input_dict"][robot_node].shape

(87, 981, 6)

In [12]:
print(len(eval_data_dict['input_dict']))
print(len(eval_data_dict['labels']))

30
26


In [13]:
STG.node_edges_and_neighbors

{HomeC/AlHorford: defaultdict(set,
             {'AwayC-HomeC': [AwayC/DeMarcusCousins, AwayC/KostaKoufos],
              'AwayPF-HomeC': [AwayPF/QuincyAcy],
              'AwayPG-HomeC': [AwayPG/RajonRondo],
              'AwaySF-HomeC': [AwaySF/RudyGay, AwaySF/OmriCasspi],
              'AwaySG-HomeC': [AwaySG/BenMcLemore],
              'HomeC-HomeC': [HomeC/TiagoSplitter],
              'HomeC-HomePG': [HomePG/DennisSchroder],
              'HomeC-HomeSG': [HomeSG/KyleKorver, HomeSG/JustinHoliday]}),
 AwayC/DeMarcusCousins: defaultdict(set,
             {'AwayC-AwayPF': [AwayPF/QuincyAcy],
              'AwayC-AwayPG': [AwayPG/RajonRondo],
              'AwayC-AwaySF': [AwaySF/RudyGay, AwaySF/OmriCasspi],
              'AwayC-AwaySG': [AwaySG/BenMcLemore],
              'AwayC-HomeC': [HomeC/AlHorford, HomeC/TiagoSplitter],
              'AwayC-HomePF': [HomePF/PaulMillsap],
              'AwayC-HomePG': [HomePG/ShelvinMack, HomePG/DennisSchroder],
              'AwayC-HomeSF': [Ho

In [14]:
%memit

peak memory: 356.74 MiB, increment: 0.00 MiB


# Training

In [14]:
# JUST RUN 'cd memory_usage_info; python training_memory_usage.py'